In [1]:
import cufflinks as cf
import numpy as np
import pandas as pd

import folium
from urllib.request import urlopen
from folium import plugins
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

#pio.renderers.default = "svg"
cf.set_config_file(theme='white')

from plotly.subplots import make_subplots

In [2]:
baixas_all = pd.read_csv('../dataframes/baixas_all.csv')
baixas_distrito = pd.read_csv('../dataframes/baixas_distrito.csv')
df_layoff = pd.read_csv('../dataframes/df_layoff.csv')
df_companies = pd.read_csv('../dataframes/companies_gender_data.csv')
df_work = pd.read_csv('../dataframes/df_work.csv')
df_size = pd.read_csv('../dataframes/organization_dimension.csv')
df_region = pd.read_csv('../dataframes/organization_region.csv')
despedimentos = pd.read_csv('../dataframes/despedimentos_coletivos.csv')
historical_data_company = pd.read_csv('../original_files/historical_data_company.csv')

### Nº de Baixas Por Isolamento

In [3]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=baixas_all['DATA'],
                         y=baixas_all['POR DIA'],
                         name='POR DIA',
                         line=dict(color='crimson')))

fig.add_trace(go.Bar(x=baixas_all['DATA'],
                     y=baixas_all['ACUMULADOS'],
                     name='ACUMULADOS',
                     marker_color='rgba(195, 206, 350, 0.9)'))

fig.update_layout(title="Nº De Baixas Por Isolamento", title_x=0, xaxis_title="",yaxis_title="Nº Prestações Requeridas",
                 width=900, height=500,template='gridon')

#plot(fig, filename='baixas_por_isolamento.html')
fig.show()

Súbidas nos dias 29 de Maio e 1 de Junho:
* No dia 29 de Maio o estado de calamidade em Portugal é renovado por mais quinze dias.
* No mesmo dia é  aprovada a terceira fase do plano de desconfinamento, com restrições e regras especiais para a área de Lisboa, devido ao aumento de casos de covid-19. 
* Nesta última fase iniciada no dia 1 de junho reabriram as grandes superfícies - as lojas com mais de 400 metros quadrados - como os centros comerciais, o pré-escolar, os centros de ocupação de tempos livres e os ginásios.
* Teletrabalho deixa de ser obrigatório nesta terceira fase. 
***
O número elevado de baixas nos dias 29 e 1 de Junho refleta o estado de preocupação da população que continua a sentir-se mais segura em casa, evitando a deslocação para o trabalho. É provável que estas baixas se notem principalmente nas grandes superfícies (centros comerciais), ginásios e pré-escolares uma vez que estas mesmas reabriram na terceira fase. "Dentro deste grupo de pessoas, encontram-se os trabalhadores dos grupos considerados de risco, que decidam faltar ao trabalho por não terem apoio previsto entre o conjunto das medidas aprovadas pelo Governo, segundo advogados consultados pela Lusa."

***
https://expresso.pt/coronavirus/2020-04-06-Covid-19-Pessoas-de-risco-que-decidam-faltar-ao-trabalho-nao-tem-apoios

In [4]:
baixas_distrito.drop([0,0], axis=0, inplace=True)

In [29]:
fig = go.Figure(go.Bar(x=baixas_distrito['TOTAL'],y=baixas_distrito['DISTRITO'],orientation='h'))

fig.update_layout(title='Nº De Baixas Por Isolamento - Por Distrito ',title_x=0, template='plotly_white', 
                  xaxis_title="Prestações Entradas por Distrito",yaxis_title="",
                 yaxis={'categoryorder':'total ascending'},width=900, height=500)
fig.show()

* Desde o início da pandemia Ovar, em Aveiro, tem sido um dos concelhos mais afectados pelo covid-19 com 690 casos confirmados até hoje (15 de Julho) entre os seus cerca de 55.400 habitantes. Trabalhadores estiveram impedidos de trabalhar no concelho ou fora dele devido à quarentena geográfica iniciada a 18 de março nesse município, na sequência do estado de calamidade pública gerado pela covid-19.

https://zap.aeiou.pt/trabalhadores-ovar-seguranca-social-cerco-sanitario-319104

* De seguida, Porto, Lisboa e Braga destacam-se entre os distritos com mais risco. 

### Lay Off - Estimativa

In [6]:
df_layoff['REMUNERAÇÕES DECLARADAS'] = df_layoff['REMUNERAÇÕES DECLARADAS'].astype(int)

In [30]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=df_layoff['DATA'], y=df_layoff['Nº TRABALHADORES'], name='Nº TRABALHADORES',
              mode = 'markers+lines')
)
fig.update_layout(height=500, width=900, title_text="Lay Off - Estimativa", title_x=0,
                 yaxis_title="Nº Trabalhadores",xaxis_title="",template='plotly_white')

fig.show()

Número de pedidos acumulados, sem indicação das empresas que já não estão em sistema de lay off. Esperava-se uma queda depois de 1 de Junho com o início da terceira fase de desconfidamento mas tal não é visível no gráfico.

### Layoff – Estim. - CAE,Dim,Dist

3.1  Data Analysis by Work Sectors and Genders

In [8]:
df=pd.read_csv('../original_files/historical_data_company.csv')
df=df.iloc[:,1:]
df_2=df.head(21)
df_unpivoted = df_2.melt(id_vars=['Setor'], var_name='Date', value_name='Total')

In [9]:
df = df_unpivoted
fig = px.bar(df, x="Date", y="Total", color='Setor',
             height=700,
             width=1000,
             title='',
            color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(template='simple_white', legend_orientation="h",xaxis=dict(type="category", categoryorder="category ascending", tickangle = 90,
        title_text = ""),yaxis = dict(
        title_text = "Nº Trabalhadores"),legend=dict(
        x=0.0,
        y=-0.3))
fig.update_xaxes(ticks="outside", tickwidth=1, ticklen=10, tickangle=45)

fig.show()

In [10]:
#Edit information to obtain the 12 most affected sectors, and include an extra sector which adds the rest of the 9 sectors, who
#were included in the study, adding them as "Outros Setores"

#Select the needed information
df_pieTotal=df_work[['Setor','Nº TRABALHADORES']]
#sort values by the number of unemployed males
df_pieTotal=df_pieTotal.sort_values(by=['Nº TRABALHADORES'],ascending=False)
#calculate the sum of the other sectors affected
TotalOutros_setores=sum(df_pieTotal['Nº TRABALHADORES'].tail(9))
#Make a new row with the sum of the rest of sectors affected
new_row=pd.Series(data={'Setor':'Outros setores','Nº TRABALHADORES':TotalOutros_setores})
#Append the new row to the existing information
df_pieTotal=df_pieTotal.append(new_row, ignore_index=True)
#sort values again by number of unemployed 
df_pieTotal=df_pieTotal.sort_values(by=['Nº TRABALHADORES'],ascending=False)
#clean the data
df_pieTotal=df_pieTotal.reset_index()
df_pieTotal=df_pieTotal.set_index('Setor')
df_pieTotal=df_pieTotal.drop('index',1)
#select the data to plot and store it in a new dataframe, select the first 12 rows (12 sectors more affected + other sectors)
df_pieTotal1=df_pieTotal['Nº TRABALHADORES']
df_pieTotal1=df_pieTotal1.head(13)

#Make a bar chart of the 12 most affected sectors + other sectors for the last report

fig = go.Figure(go.Bar(
            x=df_pieTotal1.index,
            y=df_pieTotal1.values,
            orientation='v'),
               )
fig.update_layout(
    height=800, width=1000,
    title='Nº Trabalhadores Em Layoff - Por Setor ( Top 12), Último Ficheiro',
    template='plotly_white', 
    yaxis_title="Nº Trabalhadores Em Layoff",
    xaxis_title="",
    yaxis={'categoryorder':'total ascending'}
    )
fig.update_xaxes(ticks="outside",tickangle=45,automargin=True)
fig.show()

In [11]:
#Present the same information on a pie chart
df = df_pieTotal1
fig = px.pie(df, values=df_pieTotal1.values, names=df_pieTotal1.index,
             title='% Trabalhadores Em Layoff - Por Setor ( Top 12), Último Ficheiro',width=1100, height=500)
fig.update_traces(textposition='inside', textinfo='percent')
fig.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')
fig.show()

In [12]:
# Make a list with the 12 most affected sectors, plus presenting an 13th sector with the sum of the other sectors affected  for 
#For male gender
#Select the needed information
df_pieMale=df_work[['Setor','Masculino']]
#sort values by the number of unemployed males
df_pieMale=df_pieMale.sort_values(by=['Masculino'],ascending=False)
#calculate the sum of the other sectors affected
TotalOutros_setores=sum(df_pieMale['Masculino'].tail(9))
#Make a new row with the sum of the rest of sectors affected
new_row=pd.Series(data={'Setor':'Outros setores','Masculino':TotalOutros_setores})
#Append the new row to the existing information
df_pieMale=df_pieMale.append(new_row, ignore_index=True)
#sort values again by number of unemployed 
df_pieMale=df_pieMale.sort_values(by=['Masculino'],ascending=False)
#clean the data
df_pieMale=df_pieMale.reset_index()
df_pieMale=df_pieMale.set_index('Setor')
df_pieMale=df_pieMale.drop('index',1)
#select the data to plot and store it in a new dataframe, select the first 12 rows (12 sectors more affected + other sectors)
df_pieMale=df_pieMale['Masculino']
df_pieMale=df_pieMale.head(13)


# Make a list with the 12 most affected sectors, plus presenting an 13th sector with the sum of the other sectors affected  for 
#For female gender
#Select the needed information
df_pieFemale=df_work[['Setor','Feminino']]
#sort values by the number of unemployed males
df_pieFemale=df_pieFemale.sort_values(by=['Feminino'],ascending=False)
#calculate the sum of the other sectors affected
TotalOutros_setores=sum(df_pieFemale['Feminino'].tail(9))
#Make a new row with the sum of the rest of sectors affected
new_row=pd.Series(data={'Setor':'Outros setores','Feminino':TotalOutros_setores})
#Append the new row to the existing information
df_pieFemale=df_pieFemale.append(new_row, ignore_index=True)
#sort values again by number of unemployed 
df_pieFemale=df_pieFemale.sort_values(by=['Feminino'],ascending=False)
#clean the data
df_pieFemale=df_pieFemale.reset_index()
df_pieFemale=df_pieFemale.set_index('Setor')
df_pieFemale=df_pieFemale.drop('index',1)
#select the data to plot and store it in a new dataframe, select the first 12 rows (12 sectors more affected + other sectors)
df_pieFemale=df_pieFemale['Feminino']
df_pieFemale=df_pieFemale.head(13)

In [13]:
#Present a bar chart presenting the total amount of layoffs divided by gender on separate bars.
pd_bar=pd.merge(left=df_pieMale, right=df_pieFemale, left_on=df_pieMale.index, right_on=df_pieFemale.index)

fig = pd_bar.iplot(asFigure=True, kind='bar',barmode = 'group', x='key_0',
               xTitle='',yTitle='Nº Trabahadores Despedidos',title='Nº Trabalhadores Despedidos - Por Sexo')
# plot figure
iplot(fig)

In [14]:
#Make a pie chart showing male and female layoff amount by sector
trace1 = go.Pie(labels=df_pieMale.index, 
                    values=df_pieMale.values, 
                    name= "male", 
                    hole= .5, 
                    domain= {'x': [0, .48]})

trace2 = go.Pie(labels=df_pieFemale.index, 
                    values=df_pieFemale.values, 
                    name="female", 
                    hole= .5,  
                    domain= {'x': [.52, 1]})

layout = dict(title= '% Trabalhadores Em Layoff - Por Setor/Sexo', width=1000, height=750,
              font=dict(size=10), legend=dict(orientation="h"),
                  annotations = [
                      dict(
                          x=.18, y=.5,
                          text='Masculino', 
                          showarrow=False,
                          font=dict(size=20)),
                      dict(
                          x=.82, y=.5,
                          text='Feminino', 
                         showarrow=False,
                          font=dict(size=20))])

fig = dict(data=[trace1, trace2], layout=layout)
iplot(fig)

3.2 Analysis by Organization dimension

In [15]:
df_CompSize=pd.read_csv(r'../original_files/historical_data_company_size.csv')
df_CompSize=df_CompSize.drop(['Unnamed: 0'],axis=1)
df_CompSize=df_CompSize.transpose()
df_CompSize=df_CompSize.reset_index()
df_CompSize.columns=df_CompSize.iloc[0]
df_CompSize=df_CompSize.iloc[1:]

cf.set_config_file(theme='white')
fig = df_CompSize.iplot(asFigure=True, kind='bar',barmode = 'stack', x='TOTAL',
               xTitle='Data',yTitle='Valor acumulado por tamanho',title='Trabalhadores despedidos')
fig.update_layout(xaxis=dict(type="category", categoryorder="category ascending"))
fig.update_xaxes(ticks="outside", tickwidth = 1, tickangle=45)
# plot figure
iplot(fig)

3.3 Data Analysis By Region

In [16]:
#ANALYSIS BY REGION
import json

#import the Portugal Json file
with urlopen('https://raw.githubusercontent.com/deldersveld/topojson/master/countries/portugal/portugal-districts.json') as response:
    portugal_geo = json.load(response)

#Get latitude and longitude of Portugal
Portugalcoordinates=(39.399,-8.224)

#Divide into 2 sections, continental and islands
df_continental=df_region.iloc[0:18]
df_island=df_region.iloc[18:22]

df_continental['Latitude']=['40.644','38.015','41.503','41.805','39.823','40.205','38.566','37.019','40.537','39.744','38.716','39.291',
                         '41.149','39.135','38.524','41.644','41.295','40.657']
df_continental['Longitude']=['-8.6455','-7.865','-8.420','-6.757','-7.491','-8.419','-7.900','-7.932','-7.265','-8.807','-9.133',
                            '-7.432','-8.610','-8.405','-8.893','-8.830','-7.746','-7.914']
df_continental['Latitude']=df_continental['Latitude'].astype(float)
df_continental['Longitude']=df_continental['Longitude'].astype(float)
df_continental=df_continental.reset_index()
df_continental=df_continental.drop('index',1)

In [17]:
from folium import plugins
# creation of the choropleth
map2 = folium.Map(location=Portugalcoordinates, zoom_start=6)
map2.choropleth(geo_data = portugal_geo, 
                  name='choroplet',
                  data = df_continental,
                  columns = ['Region', 'N° Empresas'],
                  topojson='objects.PRT_adm1',
                  key_on = 'feature.properties.NAME_1',
                  fill_color = 'YlOrRd', 
              fill_opacity = 0.8, 
              line_opacity = 0.3,
              legend_name = 'count by district')

#create marker clusters
for i in range(df_continental.shape[0]): 
    location = [df_continental.Latitude[i],df_continental.Longitude[i]]
    tooltip  = "Region:{}<br>,Total Percent layoff Portugal (%): {}<br>, Total empresas layoff: {}<br> ".format(df_continental['Region'][i], 
                                                                                                     round(df_continental['Percentual(%)'][i],2),
                                                                                                     df_continental['N° Empresas'][i])

    folium.Marker(location,popup=tooltip).add_to(map2)



map2             
              
               

C:\Users\Ana\anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning:

The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.



In [18]:
df_island=df_island.reset_index()

#Azores
df_azores=df_island.iloc[0:1]
df_azores['Latitude']=['37.794']
df_azores['Longitude']=['-25.506']

#import the Azores Json file
with urlopen('https://raw.githubusercontent.com/georgique/world-geojson/master/areas/portugal/azores.json') as response:
    azores_geo = json.load(response)

#Get latitude and longitude of Azores
Azorescoordinates=(37.794,-25.506)

# creation of the map
map3 = folium.Map(location=Azorescoordinates, zoom_start=6)

# instantiate a feature group for the incidents in the dataframe
feature = folium.map.FeatureGroup()



for lat, lng, in zip(df_azores.Latitude,df_azores.Longitude):
    feature.add_child(
        folium.CircleMarker(
            location=[lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
             fill_opacity=0.6))
    


#create marker clusters
for i in range(df_azores.shape[0]): 
    location = [df_azores.Latitude[i],df_azores.Longitude[i]]
    tooltip  = "Region: Azores,Total Percent layoff Portugal (%): {}<br>, Total empresas layoff: {}<br> ".format(round(df_azores['Percentual(%)'][i],2),
                                                                                                     df_azores['N° Empresas'][i])

    folium.Marker(location,popup=tooltip).add_to(map3)
    
#add incidents to map
map3


In [19]:
#Madeira
df_madeira=df_island.iloc[1:2]
df_madeira['Latitude']=['32.633']
df_madeira['Longitude']=['-16.900']
df_madeira=df_madeira.reset_index()
df_madeira=df_madeira.drop('index',1)

#import the Madeira Json file
with urlopen('https://raw.githubusercontent.com/georgique/world-geojson/master/areas/portugal/madeira.json') as response:
    madeira_geo = json.load(response)

#Get latitude and longitude of Azores
Madeiracoordinates=(32.633,-16.900)

# creation of the map
map4 = folium.Map(location=Madeiracoordinates, zoom_start=7)

# instantiate a feature group for the incidents in the dataframe
feature = folium.map.FeatureGroup()



for lat, lng, in zip(df_madeira.Latitude,df_azores.Longitude):
    feature.add_child(
        folium.CircleMarker(
            location=[lat, lng],
            radius=5, 
            color='yellow',
            fill=True,
            fill_color='blue',
             fill_opacity=0.6))
    


#create marker clusters
for i in range(df_madeira.shape[0]): 
    location = [df_madeira.Latitude[i],df_madeira.Longitude[i]]
    tooltip  = "Region: Madeira,Total Percent layoff Portugal (%): {}<br>, Total empresas layoff: {}<br> ".format(round(df_madeira['Percentual(%)'][i],2),
                                                                                                     df_madeira['N° Empresas'][i])

    folium.Marker(location,popup=tooltip).add_to(map4)
    
#add incidents to map
map4


### Redução de Actividade TI e MOE

4.1 Redução de Atividade por Dia

In [20]:
red_byday=pd.read_csv('../dataframes/reducao_atividade_porDia.csv') 

In [21]:
# Trabalhadores Independentes - Paragem Total 
df_pt=red_byday[['DATA','TI_ParagemTotal']]
df_pt['TI_ParagemTotal'] = df_pt['TI_ParagemTotal'].cumsum()
df_pt.set_index('DATA',inplace=True)
fig = go.Figure()

# Trabalhadores Independentes - Redução Atividade 
df_ra=red_byday[['DATA','TI_Reducao']]
df_ra['TI_Reducao'] = df_ra['TI_Reducao'].cumsum()
df_ra.set_index('DATA',inplace=True)
df_ra.dropna(inplace=True)

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=df_pt.index, y=df_pt['TI_ParagemTotal'].values, 
               name='Trabalhadores Independentes - Números de Pedidos para Paragem Total',mode = 'markers+lines',
                             line=dict(shape='linear'),
                             connectgaps=True),
    row=1, col=1, 
)

fig.add_trace(
    go.Scatter(x=df_ra.index, y=df_ra['TI_Reducao'].values, 
               name='Trabalhadores Independentes - Números de Pedidos para Redução de Atividade ',
               mode = 'markers+lines',
               line=dict(shape='linear'),
               connectgaps=True),
    row=1, col=2
)

fig.update_layout(height=500, width=1100, title_text="Redução de Actividade TI (esquerda) e MOE (direita)",
                  xaxis_title="Data", yaxis_title="Valor Acumulado", template='plotly_white', legend=dict(
        x=0.0,
        y=-0.4))
fig.show()

4.2. Redução de Atividade por Distrito

In [22]:
red_bydistrict=pd.read_csv('../dataframes/reducao_atividade_porDistrito_Total.csv')

In [23]:
df=red_bydistrict.iloc[1:]
df.columns=['Distritos','Prorrogação Membro Orgão Estatutário','Prorrogação Trabalhador Independente','Membro Orgão Estatutário','Trabalhador Independente']

fig = go.Figure(go.Bar(
            x=df['Trabalhador Independente'],
            y=df['Distritos'],
            orientation='h'))
fig.update_layout(
    title='Pedidos de Redução de Atividade por Distrito - Trabalhadores Independentes',
    template='plotly_white',
     xaxis_title="Pedidos",
    yaxis_title="Distrito",
    yaxis={'categoryorder':'total ascending'}
)
fig.show()

In [24]:
fig = go.Figure(go.Bar(
            x=df['Membro Orgão Estatutário'],
            y=df['Distritos'],
            orientation='h'))
fig.update_layout(
    title='Pedidos de Redução de Atividade por Distrito - Membro Orgão Estatutário',
    template='plotly_white',
     xaxis_title="Pedidos",
    yaxis_title="Distrito",
    yaxis={'categoryorder':'total ascending'}
)
fig.show()

4.3 Redução de Atividade por Sexo

In [25]:
reducao_atividade_bysex=pd.read_csv('../dataframes/reducao_atividade_porSexo.csv')
red_bysex_aux=reducao_atividade_bysex[['TipoPedido','Feminino','Masculino']]
red_bysex_aux.loc[:,'TipoPedido']=['Trabalhador Independente','Prorrog. Trabalhador Independente','Membro Orgão Estatutário','Prorrog. Membro Orgão Estat.']
# red_bysex_aux=red_bysex_aux.melt('TipoPedido', var_name='Sexo', value_name='Numero Trabalhadores')
red_bysex_aux.set_index('TipoPedido',inplace=True)
labels=['Trabalhador Independente','Prorrog. Trabalhador Independente','Membro Orgão Estatutário','Prorrog. Membro Orgão Estat.']

fig = go.Figure(data=[
    go.Bar(name='Masculino', x=labels, y=red_bysex_aux['Masculino']),
    go.Bar(name='Feminino', x=labels, y=red_bysex_aux['Feminino'])
])
# Change the bar mode
fig.update_layout(barmode='group',
                  yaxis_title="Número de Pedidos",template='plotly_white',
                 title='Pedidos de Redução de Atividade por Sexo')
fig.show()


### Despedimentos coletivos

In [26]:
# Read csv
despedimentos=pd.read_csv('../dataframes/despedimentos_coletivos.csv')

In [27]:
df = despedimentos[['DATA','COLETIVOS_TOTAL','COLETIVOS_MICRO']]
df.columns=['DATA','Total','Micro Empresas']

fig = df.iplot(asFigure=True, kind='bar', barmode = 'overlay', x='DATA',
               xTitle='Data',yTitle='Valor acumulado ao longo do mês',title='Processos de despedimento coletivos')
iplot(fig)

In [28]:
df = despedimentos[['DATA','TRABALHADORES_TOTAL','TRABALHADORES_MICRO']]
df.columns=['DATA','Total','Micro Empresas']

fig = df.iplot(asFigure=True, kind='bar',barmode = 'overlay', x='DATA',
               xTitle='Data',yTitle='Valor acumulado ao longo do mês',title='Trabalhadores a despedir')

iplot(fig)